<a href="https://colab.research.google.com/github/mnocerino23/Wildfire-Forecaster/blob/main/Regression_SmallDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As the classification models have had some difficulties classifying the fires accurately, we will attempt to build regression models (on Acres Burned feature) instead and see if these have more success.

I will deploy the following techniques:

1.   One-Hot Encoding of Categorical Variables
2.   Splitting the Training and Testing data
3.   Normalizing Data
4.   Feature Selection
5.   Regressions




In [1066]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

#Read in the smaller of the final datasets. The dataset contains around 1200 fires from 2016-2019
wildfire_set2 = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/Wildfires/wildfire_set2_w_allfeatures.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1067]:
wildfire_set2.head()

,Unnamed: 0,Year,Name,AcresBurned,Fire Size Rank,Cause,SOURCE_REPORTING_UNIT_NAME,DaysBurn,Discovery Month,Discovered DOY,...,DX90_2M,DP10_2M,Receives Snow,Snow Station,River Basin,Mar_SP,Mar_WC,Mar_Dens,Has_Elevation,Elevation
0,0,2016,Soberanes Fire,132127.0,G,NaN,NaN,83.0,Jul,NaN,...,0.0,1.0,0,NaN,NaN,0.0,0.0,0.00,1,961.2744
1,1,2016,Erskine Fire,48019.0,G,NaN,NaN,18.0,Jun,NaN,...,15.0,4.0,1,mineral_king,Kaweah,36.0,16.0,0.44,1,3389.0664
2,2,2016,Chimney Fire,46344.0,G,NaN,NaN,24.0,Aug,NaN,...,43.0,0.0,0,NaN,NaN,0.0,0.0,0.00,1,1049.8560
3,3,2016,Blue Cut Fire,36274.0,G,NaN,NaN,7.0,Aug,NaN,...,43.0,0.0,0,NaN,NaN,0.0,0.0,0.00,1,4192.8624
4,4,2016,Gap Fire,33867.0,G,NaN,NaN,1.0,Aug,NaN,...,0.0,2.0,1,parks_creek,Shasta,77.0,34.0,0.44,1,3244.7112


In [1068]:
print(wildfire_set2.columns)

Index(['Unnamed: 0', 'Year', 'Name', 'AcresBurned', 'Fire Size Rank', 'Cause',
       'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn', 'Discovery Month',
       'Discovered DOY', 'Contained Month', 'Contained DOY', 'Latitude',
       'Longitude', 'County', 'CountyIds', 'State', 'OWNER_DESCR',
       'NOAA Station', 'Link', 'AWND', 'CLDD', 'DP10', 'DX90', 'PRCP', 'TAVG',
       'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M', 'DP10_2M',
       'Receives Snow', 'Snow Station', 'River Basin', 'Mar_SP', 'Mar_WC',
       'Mar_Dens', 'Has_Elevation', 'Elevation'],
      dtype='object')


Drop all columns that will not be relevant for our regression task:Unnamed: 0, Year, Name, Cause, SOURCE_REPORTING UNIT NAME, DaysBurn, Discovered DOY, Contained Month, Contained DOY, Lat, Long, County Ids, State, OWNER_DESCR, NOAA Station, Link, Snow Station, River Basin

In [1069]:
wildfire_set2.drop(columns = ['Unnamed: 0', 'Year', 'Name', 'Cause', 'Fire Size Rank',
                      'SOURCE_REPORTING_UNIT_NAME', 'DaysBurn','Contained Month', 'Discovered DOY',
                      'Contained DOY','Latitude','Longitude','County','CountyIds','State','OWNER_DESCR',
                      'NOAA Station', 'Link', 'Snow Station', 'River Basin','Has_Elevation'], inplace = True)

Taking a look at the dataset now that we have dropped the nonrelevant columns

In [1070]:
wildfire_set2.head(5)

,AcresBurned,Discovery Month,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,PRCP_RS,DX90_2M,DP10_2M,Receives Snow,Mar_SP,Mar_WC,Mar_Dens,Elevation
0,132127.0,Jul,6.5,0.0,0.0,0.0,0.00,58.8,65.2,52.4,14.11,21.42,0.0,1.0,0,0.0,0.0,0.00,961.2744
1,48019.0,Jun,6.7,529.0,0.0,22.0,0.00,82.6,96.6,68.6,4.68,4.88,15.0,4.0,1,36.0,16.0,0.44,3389.0664
2,46344.0,Aug,6.9,237.0,0.0,23.0,0.00,72.6,92.6,52.6,2.52,8.09,43.0,0.0,0,0.0,0.0,0.00,1049.8560
3,36274.0,Aug,6.5,455.0,0.0,28.0,0.00,79.7,94.6,64.7,3.41,6.45,43.0,0.0,0,0.0,0.0,0.00,4192.8624
4,33867.0,Aug,4.5,0.0,0.0,0.0,0.02,56.4,62.9,49.9,18.03,54.17,0.0,2.0,1,77.0,34.0,0.44,3244.7112


In [1071]:
wildfire_set2.shape

(1156, 19)

# Investigate the presence of null values in the dataset:
Below, we have the count of totall null values in each column for  wildfire_set2.

In [1072]:
print(wildfire_set2.isnull().sum())

AcresBurned         3
Discovery Month     0
AWND               37
CLDD               37
DP10               32
DX90               34
PRCP               32
TAVG               37
TMAX               34
TMIN               37
PRCP_6M             7
PRCP_RS            10
DX90_2M             2
DP10_2M             1
Receives Snow       0
Mar_SP              0
Mar_WC              0
Mar_Dens            0
Elevation           0
dtype: int64


In [1073]:
wildfire_set2 = wildfire_set2.dropna()

In the cell above, we drop all null values in the dataset. We are able to drop all nulls in this case because our dataset is sizeable with relatively few null values in each so not much data is lost in this process. Looking at the shape of the dataframe after the drops, we see that the dataset remains a similar size post drop.

In [1074]:
#we lose around 50 values when dropping NaN's
wildfire_set2.shape

(1105, 19)

# For our regressions, we will try to predict the number of Acres Burned in the fire

In [1075]:
print(wildfire_set2.columns)

Index(['AcresBurned', 'Discovery Month', 'AWND', 'CLDD', 'DP10', 'DX90',
       'PRCP', 'TAVG', 'TMAX', 'TMIN', 'PRCP_6M', 'PRCP_RS', 'DX90_2M',
       'DP10_2M', 'Receives Snow', 'Mar_SP', 'Mar_WC', 'Mar_Dens',
       'Elevation'],
      dtype='object')


Double check our datatypes before we proceed with preprocessing and model building. As we can see, all features besides discovery and fire size rank are numerical (of type float or int) so all we have to do is one hot encode the discovery month of the fire.

In [1076]:
wildfire_set2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1105 entries, 0 to 1152
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   AcresBurned      1105 non-null   float64
 1   Discovery Month  1105 non-null   object 
 2   AWND             1105 non-null   float64
 3   CLDD             1105 non-null   float64
 4   DP10             1105 non-null   float64
 5   DX90             1105 non-null   float64
 6   PRCP             1105 non-null   float64
 7   TAVG             1105 non-null   float64
 8   TMAX             1105 non-null   float64
 9   TMIN             1105 non-null   float64
 10  PRCP_6M          1105 non-null   float64
 11  PRCP_RS          1105 non-null   float64
 12  DX90_2M          1105 non-null   float64
 13  DP10_2M          1105 non-null   float64
 14  Receives Snow    1105 non-null   int64  
 15  Mar_SP           1105 non-null   float64
 16  Mar_WC           1105 non-null   float64
 17  Mar_Dens      

# Below we use the describe function to get a general outlook on mean, max, min, and percentiles for each of the numerical features in the dataset

In [1077]:
wildfire_set2.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
AcresBurned,1105.0,5205.014480,31284.394585,0.0000,32.0000,87.0000,370.0000,410203.000
AWND,1105.0,6.550679,1.735019,1.3000,5.4000,6.5000,7.6000,15.000
CLDD,1105.0,279.669683,216.201422,0.0000,77.0000,272.0000,432.0000,1005.000
DP10,1105.0,0.509502,1.296648,0.0000,0.0000,0.0000,0.0000,12.000
DX90,1105.0,14.961991,11.369550,0.0000,3.0000,16.0000,26.0000,31.000
PRCP,1105.0,0.231213,0.806560,0.0000,0.0000,0.0000,0.1000,10.460
TAVG,1105.0,72.610679,9.188062,34.0000,66.0000,73.8000,79.3000,97.400
TMAX,1105.0,87.442172,11.012137,49.3000,80.1000,89.6000,95.7000,111.600
TMIN,1105.0,57.779186,8.497258,18.7000,52.7000,57.9000,63.4000,85.000
PRCP_6M,1105.0,11.244308,10.023728,0.0000,4.2000,8.8400,14.5800,67.970


# Some Interesting Takeaways:
*   Acres Burned has an average of 5200 acres yet the 50th percentile fire only has 87 acres burned. This tells us there are some extremely massive fires in this dataset contributing to this large mean acres burned
*   DP10 and PRCP look to not be the best distinguishing features as 0-50th percentile for both all have 0. This can be attributed to the vast majority of the state of California having undergone extreme drought conditions during the last decade (with some exceptions in rainy coastal areas in Northern California)
*  The temperature features seem to have a wide spread which is good
*  Three of our engineered weather features DX90_2M, PRCP_6M, PRCP_RS seem to have a healthy spread and diverse values which is good
*   Average Elevation that fires in the dataset occurred at is 1760 ft. with 75% occuring under 2398 ft. Interesting to see that most fires begin at lower elevations







# One-Hot Encoding Categorical Variables


*   Encode the discovery month (the only categorical variable) in both datasets



In [1078]:
#Add the dummy columns to the dataset
dummy_month = pd.get_dummies(wildfire_set2['Discovery Month'])
wildfire_set2 = pd.merge(left = wildfire_set2, right = dummy_month, left_index = True, right_index = True)
wildfire_set2 = wildfire_set2.drop(columns = ['Discovery Month'])
wildfire_set2.head(10)

,AcresBurned,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,...,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
0,132127.0,6.5,0.0,0.0,0.0,0.00,58.8,65.2,52.4,14.11,...,0,0,0,1,0,0,0,0,0,0
1,48019.0,6.7,529.0,0.0,22.0,0.00,82.6,96.6,68.6,4.68,...,0,0,0,0,1,0,0,0,0,0
2,46344.0,6.9,237.0,0.0,23.0,0.00,72.6,92.6,52.6,2.52,...,0,0,0,0,0,0,0,0,0,0
3,36274.0,6.5,455.0,0.0,28.0,0.00,79.7,94.6,64.7,3.41,...,0,0,0,0,0,0,0,0,0,0
4,33867.0,4.5,0.0,0.0,0.0,0.02,56.4,62.9,49.9,18.03,...,0,0,0,0,0,0,0,0,0,0
5,29322.0,6.0,629.0,0.0,31.0,0.00,85.3,99.2,71.4,2.15,...,0,0,0,0,0,0,0,0,0,0
6,12518.0,6.3,53.0,0.0,3.0,0.00,64.1,76.2,52.0,3.84,...,0,0,0,0,0,0,0,0,0,1
7,8110.0,6.5,455.0,0.0,28.0,0.00,79.7,94.6,64.7,3.41,...,0,0,0,0,0,0,0,0,0,0
8,7609.0,8.3,216.0,0.0,18.0,0.00,71.2,91.5,50.9,7.63,...,0,0,0,0,1,0,0,0,0,0
9,7474.0,5.6,19.0,0.0,0.0,0.01,63.1,71.0,55.3,9.62,...,0,0,0,0,1,0,0,0,0,0


In [1079]:
#Radomly Shuffle the dataframe to get rid of any pre-existing organization based on size
wildfire_set2 = wildfire_set2.sample(frac=1).reset_index(drop=True)

In [1080]:
wildfire_set2.head()

,AcresBurned,AWND,CLDD,DP10,DX90,PRCP,TAVG,TMAX,TMIN,PRCP_6M,...,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep
0,44.0,8.1,277.0,0.0,11.0,0.07,73.9,88.2,59.6,10.81,...,0,0,0,0,0,0,0,0,0,0
1,31.0,5.1,674.0,0.0,30.0,0.00,86.7,103.2,70.3,14.05,...,0,0,0,1,0,0,0,0,0,0
2,33.0,6.0,274.0,0.0,21.0,0.00,74.1,91.6,56.6,7.45,...,0,0,0,0,0,0,0,0,0,1
3,17357.0,4.7,31.0,1.0,5.0,0.21,62.2,81.4,43.1,4.89,...,0,0,0,0,0,0,0,0,1,0
4,150.0,4.5,112.0,0.0,0.0,0.00,68.6,76.9,60.3,8.96,...,0,0,0,1,0,0,0,0,0,0


In [1081]:
#Write the clean, encoded, and randomly shuffled data to csv before continuing 
wildfire_set2.to_csv('wildfire2_regression_clean.csv')

Split the data into train-test sets

In [1082]:
#We will train our models using the more recent dataset

In [1083]:
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler

In [1084]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, ConfusionMatrixDisplay

Utilize an 80-20 train test split on our large dataset

In [1085]:
train_data, test_data = train_test_split(wildfire_set2, test_size = 0.2, random_state = 1)

Print the shape of training and testing datasets after the split to make sure we have done this correctly

In [1086]:
print(train_data.shape)
print(test_data.shape)

(884, 30)
(221, 30)


# Feature Selection

In [1087]:
from sklearn import linear_model

In [1088]:
from sklearn.feature_selection import SelectKBest, f_regression

In [1089]:
top_features_ordered = []

In [1090]:
train_target = train_data['AcresBurned']
train_features = train_data.drop(columns = ['AcresBurned'])

In [1091]:
test_target = test_data['AcresBurned']
test_features = test_data.drop(columns = ['AcresBurned'])

Select the 20 best features using sklearn's SelectKBest

In [1092]:
best_features = SelectKBest(f_regression, k=10)
k_best_data = best_features.fit_transform(train_features, train_target)
mask = best_features.get_support()
new_features = train_features.columns[mask]
for item in new_features:
  if item not in top_features_ordered:
    top_features_ordered.append(item)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [1093]:
count = 1
for item in top_features_ordered:
  print(count, '. ', item)
  count += 1

1 .  AWND
2 .  DX90
3 .  DP10_2M
4 .  Receives Snow
5 .  Mar_SP
6 .  Mar_WC
7 .  Dec
8 .  Jul
9 .  Jun
10 .  May


# Now we normalize the train and test sets independently

In [1094]:
train_data_selected_X = train_features[top_features_ordered]
train_data_y = train_target

test_data_selected_X = test_features[top_features_ordered]
test_data_y = test_target

In [1095]:
print('Train Features:')
print('train_data_X dimensions: ', train_data_selected_X.shape)
print('train_data_y dimensions:', train_data_y.shape, '\n')

print('Test Features:')
print('test_data_X dimensions: ', test_data_selected_X.shape)
print('test_data_y dimensions:', test_data_y.shape, '\n')

Train Features:
train_data_X dimensions:  (884, 10)
train_data_y dimensions: (884,) 

Test Features:
test_data_X dimensions:  (221, 10)
test_data_y dimensions: (221,) 



Normalize data using min-max-scalar (every feature will normalized so that range of values is 0-1) after splitting into train and test

In [1096]:
from sklearn.preprocessing import MinMaxScaler

In [1097]:
#utilize minmax scaler normalization
scaler = MinMaxScaler()

#Normalize the train, validate, and test independently after splitting
train_data_X = scaler.fit_transform(train_data_selected_X)
#train_target = scaler(train_target)

test_data_X = scaler.fit_transform(test_data_selected_X)
#train_target = scaler(test_target)

# Fit the model using our normalized features to predict Acres Burned

In [1098]:
multivariate_regression = linear_model.LinearRegression()
multivariate_regression.fit(train_data_X, train_data_y)

LinearRegression()

Model coefficients: (Beta's)

In [1099]:
multivariate_regression.coef_

array([-26874.32984386,   2117.81049147, -11800.71799227,  -1828.44103599,
         7156.0599161 ,  -9939.62652506,  22627.10007144,   8601.50054135,
         2738.38673382,   3416.26380078])

Model intercept:

In [1100]:
multivariate_regression.intercept_

13794.158092445427

In [1101]:
from sklearn import metrics
import numpy as np

In [1102]:
y_predicted = multivariate_regression.predict(test_data_X)

In [1103]:
print('Mean Squared Error: ', metrics.mean_squared_error(test_data_y, y_predicted), '\n')
print('Mean Absolute Error: ', metrics.mean_absolute_error(test_data_y, y_predicted), '\n')
print('Root Mean Squared Error: ', np.sqrt(metrics.mean_squared_error(test_data_y, y_predicted)))

Mean Squared Error:  487229436.42280996 

Mean Absolute Error:  8550.464828559452 

Root Mean Squared Error:  22073.274256956305
